In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as clr
import math
import urllib.parse
import urllib.request
import json

In [ ]:
# Use this if you want to download a file manually (trim it for ease)
"""
HH: Current hour
DR: Duur van de neerslag (in 0.1 uur) per uurvak / Precipitation duration (in 0.1 hour) during the hourly division
RH: Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
"""
deBilt = pd.read_csv('data\uurgeg_260_2011-2020.csv', parse_dates=['YYYYMMDD'], comment='#', usecols=['YYYYMMDD','HH','RH','DR'])

deBilt.drop(deBilt[(deBilt['RH'] == -1) | (deBilt['RH'] == 0) | (deBilt['RH'] == '     ')].index, inplace=True)
deBilt['DR'] = deBilt['DR'].str.strip()
deBilt['DR'] = pd.to_numeric(deBilt['DR'], 'coerce').fillna(0).astype(int)
deBilt.to_csv ('data\generated\uurgeg_260_2011-2020.csv', index = False, header=True)

In [ ]:
# Use this if you want to select a lot more data (trim it for ease)
url = "http://projects.knmi.nl/klimatologie/uurgegevens/getdata_uur.cgi"
postData = {
    'start':'1971010101',
    'end':'2021010124',
    'stns': '240',
    'vars':'RH:DR'
}
data = urllib.parse.urlencode(postData)
data = data.encode('ascii')

req = urllib.request.Request(url,data)
with urllib.request.urlopen(req) as response:
    with open('data/generated/neerslag.csv','wb') as file:
        file.write(bytes("STM,YYYYMMDD,HH,RH,DR\n",encoding='utf-8'))
        file.write(response.read())
        file.close

deBilt = pd.read_csv('data/generated/neerslag.csv', parse_dates=['YYYYMMDD'], comment='#', usecols=['YYYYMMDD','HH','RH','DR'])
deBilt.drop(deBilt[(deBilt['RH'] == -1) | (deBilt['RH'] == 0) | (deBilt['RH'] == '     ') | (deBilt['DR'] == '     ')].index, inplace=True)
deBilt['DR'] = deBilt['DR'].astype(int)
deBilt.to_csv (r'data/generated/neerslag.csv', index = False, header=True)

In [ ]:
deBilt = pd.read_csv('data/generated/neerslag.csv', parse_dates=['YYYYMMDD'], comment='#', usecols=['YYYYMMDD','HH','RH','DR'],dtype={'RH':int,'DR':int})

In [ ]:
deBilt.describe()

In [ ]:
display(deBilt.plot(x='YYYYMMDD',y='RH'))

In [ ]:
"""
Count the times a certain amount of rain occured (max time 1 hour)
"""

rainRanges = [*range(2, 10, 2), *range(10, 50, 10), *range(50, 350, 50)]
freq = {}

for year in range(deBilt['YYYYMMDD'].dt.year.min(), deBilt['YYYYMMDD'].dt.year.max()+1):
    yearRain = deBilt[deBilt['YYYYMMDD'].dt.year == year]
    counts = []
    for i, mm in enumerate(rainRanges[1:],1):
        prev = rainRanges[i-1]
        x = yearRain[(yearRain['RH'] >= prev) & (yearRain['RH'] < mm)]['RH'].count()
#         print(f'year: {year} max:{mm} count:{x}')
        counts.append(x)
    freq.update({year:counts})

fpy = pd.DataFrame.from_dict(freq, orient='index',columns=rainRanges[1:])

In [ ]:
display(fpy[0:5])

In [ ]:
x = fpy.plot(figsize=(16,14),colormap="winter")

In [ ]:
"""
HH: Current hour
DR: Duur van de neerslag (in 0.1 uur) per uurvak / Precipitation duration (in 0.1 hour) during the hourly division
RH: Uursom van de neerslag (in 0.1 mm) (-1 voor <0.05 mm) / Hourly precipitation amount (in 0.1 mm) (-1 for <0.05 mm)
"""
deBilt.sort_values(by=['RH','DR'],ascending=False)[:20]

In [ ]:
# Specifieke regen val X n maal in 50 jaar
fpy.mean()

In [ ]:
fpy.mean() * 20